In [1]:
import os, sys
import numpy as np
import pandas as pd

codepath = '/user_data/mmhender/image_stats_gabor/code/'
sys.path.insert(0,codepath)

from plotting import summary_plots, load_fits
from utils import roi_utils, default_paths

#### Load fit results
Model fit on raw data, and model fit on the semantic (coco-all) residuals 

In [2]:
subjects = np.arange(1,9)
n_subjects = len(subjects)

feature_type_list_long = ['gabor_solo_ridge_12ori_8sf',
                          'gabor_solo_ridge_12ori_8sf_from_residuals']
feature_type_list = ['gabor','gabor-residuals']

n_models = len(feature_type_list)
out_list = []
for fi, fitting_type in enumerate(feature_type_list_long):

    out = [load_fits.load_fit_results(subject=ss, fitting_type=fitting_type, \
                                      n_from_end=0, verbose=False) \
           for ss in subjects]
    out_list.append(out)
    
out = out_list
    
fig_save_folder=None

# create ROI definitions
roi_def = roi_utils.multi_subject_roi_def(subjects, remove_ret_overlap=True, \
                                          remove_categ_overlap=True)
roi_names =roi_def.roi_names
n_rois = roi_def.n_rois

n_vox_each_subj = [roi_def.ss_roi_defs[si].retlabs.shape[0] for si in range(n_subjects)]
subject_inds = np.concatenate([si*np.ones((n_vox_each_subj[si],),dtype=int) \
                               for si in range(n_subjects)], axis=0)


#### Counts after thresholding by both models' R2

In [3]:
counts = np.zeros((n_subjects, n_rois),dtype=int)

for ss in range(n_subjects):
    
    # take all voxels that are above r2 threshold for both models
    abv_r2_thresh = (out[0][ss]['val_r2'][:,0]>0.01) & \
                    (out[1][ss]['val_r2'][:,0]>0.01)
    
    for rr in range(n_rois):
        
        counts[ss,rr] = np.sum(roi_def.ss_roi_defs[ss].get_indices(rr) & abv_r2_thresh)
        
counts = np.concatenate([counts, np.sum(counts, axis=0, keepdims=True)], axis=0)
roi_df = pd.DataFrame(data=counts, columns=roi_names, \
                      index=['S%02d'%ss for ss in subjects] + ['total'])

roi_df.to_csv(os.path.join(default_paths.fig_path,'roi_defs_resid_thresh.csv'))
roi_df


,V1,V2,V3,hV4,V3ab,IPS,OPA,PPA,RSC,OFA,FFA,EBA
S01,1547,1372,1183,421,204,125,460,222,74,219,263,736
S02,1092,878,965,353,200,206,531,408,296,246,386,638
S03,1243,986,796,216,176,55,82,151,71,304,156,345
S04,797,747,559,157,79,16,198,179,194,176,149,135
S05,1078,887,685,288,151,121,430,459,76,366,136,493
S06,1123,1025,858,384,78,80,228,179,20,166,79,291
S07,1062,838,516,255,37,86,314,129,13,113,98,313
S08,767,681,519,227,15,11,78,77,7,95,277,191
total,8709,7414,6081,2301,940,700,2321,1804,751,1685,1544,3142
